In [2]:
import requests
from bs4 import BeautifulSoup
import cloudscraper as cs
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, ForeignKey
import pandas as pd
import psycopg2
from dotenv import load_dotenv
import os


In [3]:
#Sectors to be excluded from the scraping
sector_to_be_excluded = ['Consumer Discretionary','Tactical Asset Allocation','Long-Term','Long/Short Broad-based','Long/Short Commodities','Long/Short Currency','Long/Short Equity','Long/Short Volatility',
                  'Low Correlation','Short-Term','Yield Curve','Value','Ultra-Short Term','Trend-Following','Consumer Discretionary' ,'Tactical Asset Allocation']

#Loading the env variable
load_dotenv()
database_connection = os.getenv('database')


In [4]:
# Function responsible to download the HTML from the URL specified
def GetPageContent(url):
    
    #Bypassing the anti-bot protection
    scraper = cs.CloudScraper()
    
    #Download the content from the URL
    content = scraper.get(url).text
    
    #Return a HTML format
    return BeautifulSoup(content, features="html.parser")

In [5]:
# Function responsible to retrieve the Sectors list 
def GetSectorsList():
    #Initialize the variables
    by_sector_actived = False
    dict_sectors = {}
    
    #Call the function to download the HTML
    response_html = GetPageContent('https://www.etf.com/channels/#Sector')
    
    #Check if there's any html content
    if response_html is not None:
        
        #Looping the HTML/CSS structure, identify the divs , which the class match exactly from the script needs to get the sector's name from
        for x in response_html.find_all('div', {'class': 'col-md-6 list-block'}):
            
            #Retrieve all the href(Links), it's how the sectors is set up
            a_hrefs = x.find_all(href=True)
            
            #Looping in all href(Links) 
            for y in a_hrefs:                
                #Check if the interaction is in the first position of the sector list
                if y.text == 'Aerospace & Defense':
                    
                    #Set the variable as true and start reading the sectors
                    by_sector_actived = True
                
                #Skip the 2 sectors
                if y.text in sector_to_be_excluded:
                    continue
                    
                #Check if the interaction has already started reading the first position    
                if y.text == 'Aerospace & Defense' or by_sector_actived == True:
                    
                    #Get the ETF 
                    dict_sectors[y.text] = f"https://www.etf.com/{y['href']}"                    
                    
                    #Check if the interaction is in the last position
                    if y.text == 'Utilities':                      
                        
                        if len(dict_sectors) > 0:
                            return dict_sectors
                        else:
                            return None


In [6]:
# Function responsible to retrieve all the ETF details
def GetETFDetails():
    
    #Retrieve the sector's list and the URL for each one
    sectors = GetSectorsList()
    
    #check if the list is not empty
    if len(sectors) > 0:
        etf_list = []
        
        #looping over the ETF sectors and URL
        for key, value in sectors.items():
            
            #Print the current sector that is being reading
            print(key)
            
            #Download HTML content from the ETF sector
            response_html = GetPageContent(value)
            
            #Filter the div, which match exactly with the position in the page
            txt_box = response_html.find_all('div', {'id': 'channels-txt'})
            
            
            #Looping over the data filtered in the step before
            for r in txt_box:
                
                #Filter all the href
                hrefs = r.find_all(href=True)
                
                #Get the total href found
                total_etfs_links = len(hrefs)

                #If it's greater than 2, because some sectors has more than 2 ETF's in the screen position
                if total_etfs_links > 2:
                    
                    #Retrieve the ETF information from the end of the href list
                    etf = hrefs[len(hrefs) - 2]
                    
                    #Append to the ETF list
                    etf_list.append({'ticker': str(etf['href'])[1:], 'sector': key, 'etf_name': etf.text})
                
                #If it's only 2, retrieve in the standard way as shown in the page
                elif total_etfs_links == 2:
                    etf = hrefs[0]
                    
                    #Append to the ETF list
                    etf_list.append({'ticker': str(etf['href'])[1:], 'sector': key, 'etf_name': etf.text})
                else:
                    continue
        
        #Return the list if has content
        if len(etf_list) > 0:
            return etf_list
        else:
            return None


In [7]:
#Function responsible to persist the data in the database
def PersistETFdatabase(etf_list):
    
    #Initialize the Metadata
    metadata = MetaData()
    
    #Set up the 
    engine = create_engine(database_connection)
    
    #Create the local table with the same structure as the database
    table = Table('etf_list', metadata,
                  Column('ticker', String(100), primary_key=True),
                  Column('sector', String(100)),
                  Column('etf_name', String(200)),
                  )
    #Insert the full list in one time
    engine.execute(table.insert(), etf_list)

In [8]:
#Call the function to retreive all the ETF details in a list
etf_list = GetETFDetails()

Aerospace & Defense
Aggressive
Artificial Intelligence
Autos
Bank
Banking and Investment Services
Basic Materials
Biotechnology
Bitcoin
Broad Maturities
Capital Appreciation
Casinos / Gaming
Coal
Communications Equipment
Conservative
Constant Weight
Construction & Engineering
Consumer
Consumer Cyclicals
Consumer non-Cyclicals
Derivative
Environment
Financial Services
Financials
Floating Rate
Front Month
Growth
Health Care Equipment & Supplies
Health Care Providers & Services
Healthcare
Hedge Funds
Homebuilding
In Specie
Income
Income & Capital Appreciation
Industrials
Inflation
Infrastructure
Insurance
Intermediate
Internet
Laddered
Leisure & Recreation
MLPs
Marijuana
Media & Publishing
Merger Arbitrage
Moderate
Mortgages
Oil & Gas Equipment & Services
Oil & Gas Exploration & Production
Pharmaceuticals
Phones & Handheld Devices
Physically Held
Private Equity
REIT
Real Estate
Renewable Energy
Restaurants & Bars
Retail
Semiconductors
Shipping
Software
Solar
Steel
Technology
Technology Eq

In [9]:
#Check if the list is not empty
if len(etf_list) > 0:
    
    #Persist the etf list in the database
    PersistETFdatabase(etf_list)